In [1]:
include("../../MCVI/Planner.jl")
using Plots
using RockSample

In [2]:
# pomdp = TigerPOMDP()
# pomdp = RockSamplePOMDP(15,15)
# pomdp = RockSamplePOMDP(11,11)
pomdp = RockSamplePOMDP(7,8)

action_space = actions(pomdp)
obs_space = observations(pomdp)

nb_particles_b0 = 100
max_node_size = 10000

action_space = collect(actions(pomdp))
b0 = initialstate(pomdp)
b0_particles, b0_weighted,action_space = InitPOMDP(pomdp, b0, nb_particles_b0)

# obs_ = collect(observations(pomdp))
# length(b0)
fsc = InitFSC(max_node_size, action_space, obs_space)

FSC(Dict{Pair{Any, Int64}, Int64}[Dict(), Dict(), Dict(), Dict(), Dict(), Dict(), Dict(), Dict(), Dict(), Dict()  …  Dict(), Dict(), Dict(), Dict(), Dict(), Dict(), Dict(), Dict(), Dict(), Dict()], FscNode[], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], 1:3)

In [3]:
action_space

13-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13

In [4]:
RL = FindRLower(pomdp, b0_particles, action_space)

0.0

In [5]:
beliefs = BeliefUpdate(b0_particles, rand(action_space), 100, pomdp)

(Dict{Any, Any}(2 => RSState{8}[RSState{8}([1, 1], Bool[0, 0, 1, 0, 1, 0, 0, 0]), RSState{8}([1, 1], Bool[0, 0, 1, 0, 1, 0, 0, 0]), RSState{8}([1, 1], Bool[0, 0, 0, 0, 1, 0, 0, 0]), RSState{8}([1, 1], Bool[1, 1, 0, 0, 1, 0, 1, 1]), RSState{8}([1, 1], Bool[1, 1, 1, 1, 1, 0, 0, 1]), RSState{8}([1, 1], Bool[1, 0, 1, 0, 1, 0, 1, 0]), RSState{8}([1, 1], Bool[1, 1, 1, 1, 1, 0, 0, 1]), RSState{8}([1, 1], Bool[1, 0, 1, 0, 1, 0, 1, 0]), RSState{8}([1, 1], Bool[0, 1, 0, 0, 0, 0, 1, 0]), RSState{8}([1, 1], Bool[0, 0, 1, 0, 0, 0, 1, 0])  …  RSState{8}([1, 1], Bool[1, 0, 1, 0, 1, 0, 1, 0]), RSState{8}([1, 1], Bool[0, 1, 1, 1, 1, 0, 0, 0]), RSState{8}([1, 1], Bool[1, 1, 1, 0, 1, 1, 0, 0]), RSState{8}([1, 1], Bool[1, 1, 1, 1, 1, 0, 0, 1]), RSState{8}([1, 1], Bool[1, 1, 0, 0, 0, 0, 0, 1]), RSState{8}([1, 1], Bool[1, 0, 1, 0, 1, 0, 0, 0]), RSState{8}([1, 1], Bool[1, 0, 1, 1, 1, 0, 1, 1]), RSState{8}([1, 1], Bool[1, 0, 1, 0, 0, 0, 0, 1]), RSState{8}([1, 1], Bool[1, 1, 1, 0, 1, 0, 1, 0]), RSState{8}([1, 

In [6]:
fsc

FSC(Dict{Pair{Any, Int64}, Int64}[Dict(), Dict(), Dict(), Dict(), Dict(), Dict(), Dict(), Dict(), Dict(), Dict()  …  Dict(), Dict(), Dict(), Dict(), Dict(), Dict(), Dict(), Dict(), Dict(), Dict()], FscNode[], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], 1:3)

In [7]:
# node_start = CreatNode(b0_particles, action_space, obs_space)
# push!(fsc._nodes, node_start)
# nI_start = length(fsc._nodes)

In [8]:
# BackUp(nI_start, fsc, 10, 100, pomdp, action_space, obs_space)

In [9]:
# belief_node_list = Vector{Int64}()
# ExpandBeliefs(fsc, 1, rand(b0_particles), 100, 0, 10, pomdp, action_space, obs_space,belief_node_list)

In [10]:
# belief_node_list

In [11]:
# fsc._nodes[1]._state_particles

In [12]:
# fsc._nodes[2]._state_particles

In [13]:
# rand(fsc._nodes[2]._state_particles)

In [14]:
MCVIPlanning(b0_particles, b0_weighted, fsc, pomdp, RL, 20, 300, 0.1, 200)

!! Belief Expand Process !!
adding new beliefs from node 1 with action 5
BackUp Process
nI_new 2, V 0.0
nI_new 1, V 0.0
!! Belief Expand Process !!
adding new beliefs from node 1 with action 9
BackUp Process
nI_new 4, V 0.0
nI_new 3, V 0.0
nI_new 2, V 0.0014868580853204696
nI_new 1, V 0.03156402539137917
!! Belief Expand Process !!
adding new beliefs from node 1 with action 10
BackUp Process
nI_new 6, V 0.0
nI_new 5, V 0.0
nI_new 4, V 0.0
nI_new 3, V 0.0
nI_new 2, V 0.0
nI_new 1, V 0.0
!! Belief Expand Process !!
adding new beliefs from node 1 with action 13
BackUp Process
nI_new 8, V 0.0
nI_new 7, V 0.0
nI_new 6, V 0.0
nI_new 5, V 0.0
nI_new 4, V 0.0
nI_new 3, V 0.0
nI_new 2, V 0.0
nI_new 1, V 0.0
!! Belief Expand Process !!
adding new beliefs from node 1 with action 2
BackUp Process
nI_new 9, V 0.0
nI_new 8, V 0.0
nI_new 7, V 0.0
nI_new 6, V 0.0
nI_new 5, V 0.0
nI_new 4, V 0.0
nI_new 3, V 0.0
nI_new 2, V 0.0
nI_new 1, V 0.0
!! Belief Expand Process !!
adding new beliefs from node 1 w

LoadError: InterruptException:

In [15]:
fsc._nodes[1]._Q_action

Dict{Any, Float64} with 13 entries:
  5  => 0.0
  12 => 0.0
  8  => 0.0
  1  => 0.0
  6  => 0.0
  11 => 0.0
  9  => 0.0
  3  => 0.0
  7  => 0.0
  4  => 0.0
  13 => 0.0
  2  => 0.0
  10 => 0.0

In [16]:
fsc._nodes[1]._V_node

0.0

In [17]:
fsc._eta[1]

Dict{Pair{Any, Int64}, Int64} with 39 entries:
  2=>3  => 1
  12=>3 => 1
  3=>3  => 1
  9=>1  => 1
  1=>2  => 1
  4=>3  => 1
  7=>2  => 1
  13=>3 => 1
  5=>1  => 1
  11=>1 => 1
  8=>2  => 1
  10=>2 => 1
  2=>2  => 1
  6=>1  => 1
  9=>3  => 1
  12=>2 => 1
  3=>2  => 1
  5=>3  => 1
  4=>2  => 1
  11=>3 => 1
  13=>2 => 1
  1=>1  => 1
  7=>1  => 1
  6=>3  => 1
  8=>1  => 1
  ⋮     => ⋮

In [18]:
SimulationWithFSC(b0, pomdp, fsc, 80)

---------
step: 1
state:RSState{8}([1, 1], Bool[0, 0, 1, 0, 0, 0, 0, 0])
perform action:5
recieve obs:3
nI:1
nI value:0.0
reward:0.0
---------
step: 2
state:RSState{8}([1, 1], Bool[0, 0, 1, 0, 0, 0, 0, 0])
perform action:5
recieve obs:3
nI:1
nI value:0.0
reward:0.0
---------
step: 3
state:RSState{8}([1, 1], Bool[0, 0, 1, 0, 0, 0, 0, 0])
perform action:5
recieve obs:3
nI:1
nI value:0.0
reward:0.0
---------
step: 4
state:RSState{8}([1, 1], Bool[0, 0, 1, 0, 0, 0, 0, 0])
perform action:5
recieve obs:3
nI:1
nI value:0.0
reward:0.0
---------
step: 5
state:RSState{8}([1, 1], Bool[0, 0, 1, 0, 0, 0, 0, 0])
perform action:5
recieve obs:3
nI:1
nI value:0.0
reward:0.0
---------
step: 6
state:RSState{8}([1, 1], Bool[0, 0, 1, 0, 0, 0, 0, 0])
perform action:5
recieve obs:3
nI:1
nI value:0.0
reward:0.0
---------
step: 7
state:RSState{8}([1, 1], Bool[0, 0, 1, 0, 0, 0, 0, 0])
perform action:5
recieve obs:3
nI:1
nI value:0.0
reward:0.0
---------
step: 8
state:RSState{8}([1, 1], Bool[0, 0, 1, 0, 0, 0, 0, 

In [19]:
EvaluationWithSimulationFSC(b0, pomdp, fsc, discount(pomdp), 10000)

avg sum:0.0
avg sum valid search:0.0
unvalid search:0.0


In [20]:
1 in a_set

LoadError: UndefVarError: `a_set` not defined

In [21]:
max(1,2)

2

In [22]:
fsc._eta[3]

Dict{Pair{Any, Int64}, Int64} with 39 entries:
  2=>3  => 1
  12=>3 => 1
  3=>3  => 1
  9=>1  => 1
  1=>2  => 1
  4=>3  => 1
  7=>2  => 1
  13=>3 => 1
  5=>1  => 1
  11=>1 => 1
  8=>2  => 1
  10=>2 => 1
  2=>2  => 1
  6=>1  => 1
  9=>3  => 1
  12=>2 => 1
  3=>2  => 1
  5=>3  => 1
  4=>2  => 1
  11=>3 => 1
  13=>2 => 1
  1=>1  => 1
  7=>1  => 1
  6=>3  => 1
  8=>1  => 1
  ⋮     => ⋮

In [23]:
fsc._nodes[3]._Q_action

Dict{Any, Float64} with 13 entries:
  5  => 0.0
  12 => 0.0
  8  => 0.0
  1  => 0.0
  6  => 0.0
  11 => 0.0
  9  => 0.0
  3  => 0.0
  7  => 0.0
  4  => 0.0
  13 => 0.0
  2  => 0.0
  10 => 0.0

In [24]:
# maybe it's the time not enough for computing a full policy